# Анализ и обработка отзывов

In [ ]:
import pandas as pd
from tqdm import tqdm
import pymorphy3

# Инициализация морфологического анализатора для работы с русским языком
morph_analyzer = pymorphy3.MorphAnalyzer()


class CreateDataset:
	def __init__(self):
		# Определяем ключевые слова для категорий отзывов
		self.practice = {'задачи', 'кейс', 'проект', 'задания'}  # Слова, связанные с практикой
		self.theory = {'знания', 'лекция', 'материал'}  # Слова, связанные с теорией
		self.teacher = {'лектор', 'профессор', 'наставник', 'педагог', 'препод', 'автор',
						'создатель'}  # Слова, связанные с преподавателем
		self.technology = {'ресурсы', 'инструменты', 'доступность',
						   'интерактивность'}  # Слова, связанные с технологиями
		self.relevance = {'современный', 'релевантный', 'новый', 'старый',
						  'устарел'}  # Слова, связанные с актуальностью

		# Список ключевых тем
		self.topic_keywords = ['практика', 'теория', 'преподаватель', 'технологии', 'актуальность']

		# Создаем словарь, где ключевые темы связываются с набором соответствующих слов
		self.topics = {keyword: {keyword} for keyword in self.topic_keywords}

		# Обновляем словарь ключевых тем с учетом различных словоформ
		self.topics['практика'].update(self.practice)
		self.topics['теория'].update(self.theory)
		self.topics['преподаватель'].update(self.teacher)
		self.topics['технологии'].update(self.technology)
		self.topics['актуальность'].update(self.relevance)

	def names_in_text(self, text: str) -> bool:
		"""
        Проверяет, содержатся ли в тексте имена преподавателей.
        :param text: Исходный текст
        :return: True, если найдено хотя бы одно имя, иначе False
        """
		skip_names = frozenset('паскаль')  # Исключаемое имя (например, Паскаль)
		for word in text:
			if word not in skip_names:
				parse = morph_analyzer.parse(word)[0]
				tag = parse.tag
				# Проверяем, является ли слово именем или фамилией с высокой вероятностью
				if ('Name' in tag or 'Surn' in tag) and parse.score >= 0.8:
					return True
		return False

	def automatic_annotation(self, df: pd.DataFrame) -> pd.DataFrame:
		"""
        Автоматическая разметка обучающего набора данных на основе ключевых слов и имен преподавателей.
        :param df: DataFrame с текстами для разметки
        :return: Размеченный DataFrame
        """
		# Подготовка структуры для хранения данных
		data_all = {t: [] for t in self.topic_keywords}
		data_index = set()

		# Проходим по каждому тексту в DataFrame
		for idx, row in tqdm(df.iterrows(), total=len(df)):
			preprocessed_text = row['PreprocessedText'].split()  # Предобработанный текст
			data = []
			for key, keywords in self.topics.items():
				# Проверяем наличие ключевых слов в тексте с использованием нечеткого поиска
				if any(True for keyword in keywords for word in preprocessed_text if fuzz.ratio(word, keyword) > 78):
					data_index.add(idx)
					data.append(1)
				elif key == 'преподаватель' and self.names_in_text(preprocessed_text):
					# Дополнительная проверка на наличие имен преподавателей
					data_index.add(idx)
					data.append(1)
				else:
					# Если текст не относится к категории
					data.append(0)
			if sum(data) > 0:
				# Добавляем данные в соответствующие столбцы
				for i, key in enumerate(self.topic_keywords):
					data_all[key].append(data[i])
		return pd.DataFrame(data_all, index=sorted(data_index))

## Обработка текста

In [ ]:
# Импортируем датасет с отзывами
import pandas as pd
from thefuzz import fuzz  # Новая версия библиотеки fuzzywuzzy
import nltk

# Загрузка необходОгромная благодарность авторам за труд! Полезный материал пра…имых данных для NLTK
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Загрузка стоп-слов
stop_words = stopwords.words('russian')

# Загрузка данных
df = pd.read_csv('../data/train_reviews.csv', index_col=0)
df['Reviews'] = df['Reviews'].astype(str)


class TextProcessor:
	def preprocess(self, text: str) -> str:
		"""
        Предобрабатывает текст, токенизируя, лемматизируя и удаляя стоп-слова.

        :param text: Входной текст, который необходимо предобработать.
        :return: Список предобработанных слов.
        """
		tokens = word_tokenize(text.lower())
		filtered_tokens = [
			normalized_word
			for word in tokens
			if (normalized_word := morph_analyzer.parse(self._remove_symbols(word))[0].normal_form) not in stop_words
		]

		filtered_tokens = " ".join(filtered_tokens).replace('ё', 'е').split()  # Удаление лишних пробелов
		return " ".join(filtered_tokens)

	@staticmethod
	def _remove_symbols(text: str) -> str:
		"""Удаляет все символы и цифры из строки."""
		clean_text = "".join(char if char.isalnum() and not char.isdigit() else " " for char in text)
		return clean_text

### Применяем автоматическую разметку к `train_reviews.csv`

In [ ]:
text_processor = TextProcessor()

# Применение предобработки к отзывам
df['PreprocessedText'] = df['Reviews'].apply(text_processor.preprocess)
# Создание списка всех стеммированных слов
all_words = [word for sublist in df['PreprocessedText'] for word in sublist]
unique_words = pd.Series(all_words).value_counts()

# Создание DataFrame с индексами и правильными ответами
create_dataset = CreateDataset()
df_dataset_index = create_dataset.automatic_annotation(df)

topic_keywords = create_dataset.topic_keywords

# По индексам определяем размеченные отзывы
df_marked: pd.DataFrame = df.loc[df_dataset_index.index]
for topic in topic_keywords:
	df_marked[topic] = df_dataset_index[topic]

df_marked.head()

Просмотр количества размеченных текстов

In [ ]:
import matplotlib.pyplot as plt

print('Всего размеченных текстов -', len(df_dataset_index))
for t in topic_keywords:
	print(t, '-', len(df_dataset_index[df_dataset_index[t] == 1]))

# Создание списков для тем и их количества
topics = topic_keywords
counts = [len(df_dataset_index[df_dataset_index[t] == 1]) for t in topics]

# Визуализация
plt.figure(figsize=(10, 6))
plt.bar(topics, counts, color='skyblue')
plt.xlabel('Темы')
plt.ylabel('Количество размеченных текстов')
plt.title('Количество размеченных текстов по темам')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

### Ищем отзывы с нулевыми классами и добавляем их к `df_marked`
Нулевые классы - классы не относящиеся ни к одной категории.
Нулевые классы делают обучение bert более качественным

In [ ]:
initial_void_index = list(set(df.index) - set(df_dataset_index.index))
initial_void_df = df.loc[initial_void_index]
initial_void_df = initial_void_df[
	initial_void_df['PreprocessedText'].str.split().str.len() <= 5]  # Берём строки с количеством слов <= n 

refined_void_df = pd.DataFrame()
for idx, row in initial_void_df.iterrows():
	text = row['PreprocessedText'].split()  # Предобработанный текст
	for key, keywords in create_dataset.topics.items():
		# Проверяем наличие ключевых слов в тексте с использованием нечеткого поиска
		if any(True for keyword in keywords for word in text if fuzz.ratio(word, keyword) > 67):
			break
	else:
		# Нет ключевых слов (нулевые классы)
		refined_void_df = pd.concat([refined_void_df, row.to_frame().T], ignore_index=True)
refined_void_df[topic_keywords] = 0

# Объединяем размеченные и нулевые отзывы
refined_void_df = refined_void_df[:100]  # Берём только n шт. нулевых классов
df_marked = pd.concat([df_marked, refined_void_df], ignore_index=True)

In [ ]:
df_marked.shape

## Загрузка данных

Загрузим данные, которые мы будем использовать для обучения


Посмотрим на распределение классов в выборке

## Предобработка данных

При работе с моделью BERT предобработка в традиционном смысле (удаление стоп-слов, знаков препинания) не требуется.

Нужна предобработка другого рода:
* добавление спецтокенов для разделения предложений [SEP] и классификации [CLS]
* приведение всех предложений к одинаковой длине (паддинг)
* создание маски внимания (attention mask) — списка из 0 и 1, где 0 соответствует вспомогательным токенам (padding), а 1 — настоящим.

Нам не нужно самим добавлять спецсимволы и составлять словарь соответствия токенов и индексов. Это сделает токенизатор, соотвествующий выбранной модели. Сегодня мы будем использовать модель 'bert-base-cased'.

In [ ]:
PRE_TRAINED_MODEL_NAME = 'DeepPavlov/rubert-base-cased'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, clean_up_tokenization_spaces=True)

Вспомним, как работает модель токенизации для BERT.

- Токенизируйте текст `sample_txt` и переведите токены `tokens` в индексы `token_ids`.

In [ ]:
sample_txt = df_marked.loc[0, 'Reviews']
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'Предложение: {sample_txt}')
print(f'Токены: {tokens}')
print(f'Индексы токенов: {token_ids}')

### Специальные токены

Токенизатор уже содержит индексы для спецсимволов:
- [SEP] — метка конца предложения
- [CLS] — токен для классификации предложения
- [PAD] — токен для выравнивания длин последовательностей

In [ ]:
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)

Вся предобработка может быть сделана с помощью метода `encode_plus`. Он возвращает словарь с ключами `input_ids` и `attention_mask`.

In [ ]:
encoding = tokenizer.encode_plus(
	sample_txt,  # преобразуемый текст
	max_length=256,  # максимальная длина
	add_special_tokens=True,  # добавить спецтокены [CLS] и [SEP]
	return_token_type_ids=False,  # вернуть номер предложения
	padding='max_length',  # паддинг по установленной максимальной длине
	return_attention_mask=True,  # создать маску для механизма внимания
	return_tensors='pt',  # вернуть тензор PyTorch
	truncation=True  # обрезать предложения длинее max_length
)
encoding

### Унификация длины предложений

Проанализируем, какая длина отзывов встречается в данных чаще. Отберем отзывы длины менее 512 токенов, поскольку это максимальная длина последовательности для модели BERT.

In [ ]:
token_lens = []
for txt in df_marked['Reviews'].tolist():
	tokens = tokenizer.encode(txt, max_length=512)
	token_lens.append(len(tokens))

import warnings

warnings.filterwarnings('ignore')

import seaborn as sns

sns.distplot(token_lens)
plt.xlim([0, 256])
plt.xlabel('Token count')

Установим максимальную длину последовательности равной 250.

In [ ]:
MAX_LEN = 250

### Создание датасета

Теперь создадим датасет PyTorch, который понадобится для обучения и тестирования модели.
- В методе `__init__` задаем тексты отзывов (`self.reviews`), метки классов (`self.targets`), токенизатор (`self.tokenizer`) и максимальную длину последовательности (`self.max_len`).
- В методе `__len__` определяем размер датасета.
- В методе `__getitem__` сопоставляем тексты отзывов и метки классов по индексу (`item`). Метод возвращает словарь: текст отзыва, индексы токенов, маску внимания, метку класса.

- Добавьте предобработку отзыва `review`: преобразуйте его в словарь `encoding` с помощью метода `encode_plus`, добавьте спецтокены, установите максимальную длину для паддинга, не возвращайте номер предложения, задайте паддинг по максимальной установленной длине, создайте маску для механизма внимания, установите формат списка индексов как тензор pytorch, установите усечение для предложений больше максимальной длины.

In [ ]:
from torch.utils.data import Dataset


class GPReviewDataset(Dataset):

	def __init__(self, reviews, targets, tokenizer, max_len):
		self.reviews = reviews
		self.targets = targets
		self.tokenizer = tokenizer
		self.max_len = max_len

	def __len__(self):
		return len(self.reviews)

	def __getitem__(self, item):
		review = str(self.reviews[item])
		target = self.targets[item]

		encoding = self.tokenizer.encode_plus(
			review,  # преобразуемый текст
			max_length=MAX_LEN,  # максимальная длина
			add_special_tokens=True,  # добавить спецтокены [CLS] и [SEP]
			return_token_type_ids=False,  # вернуть номер предложения
			padding='max_length',  # паддинг по установленной максимальной длине
			return_attention_mask=True,  # создать маску для механизма внимания
			return_tensors='pt',  # вернуть тензор PyTorch
			truncation=True  # обрезать предложения длинее max_length
		)
		return {
			'review_text': review,
			'input_ids': encoding['input_ids'].flatten(),
			'attention_mask': encoding['attention_mask'].flatten(),
			'targets': torch.tensor(target, dtype=torch.long)
		}

Разделим данные на обучающую, валидационную и тестовую выборки. 90% всех данных отберем для обучения, оставшиеся 10% поделим пополам для валидации и тестирования.

In [ ]:
import torch
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
df_train, df_test = train_test_split(df_marked, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

print(f'Размеры датасетов:')
print(f'Обучающая выборка: {df_train.shape}')
print(f'Валидационная выборка: {df_val.shape}')
print(f'Тестовая выборка: {df_test.shape}')

Определяем устройство для выполнения вычислений (CPU или GPU)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Создадим итераторы по данным:
- `train_data_loader` — данные для дообучения модели;
- `val_data_loader` — данные для валидации модели при обучении;
- `test_data_loader` — данные для тестирования модели.

In [ ]:
from torch.utils.data import DataLoader


def create_data_loader(df_marked, tokenizer, max_len, batch_size):
	ds = GPReviewDataset(
		reviews=df_marked['Reviews'].to_numpy(),
		targets=df_marked[topic_keywords].to_numpy(),
		# укажите, какие целевые переменные хотите использовать
		tokenizer=tokenizer,
		max_len=max_len
	)

	return DataLoader(
		ds,
		batch_size=batch_size,
		num_workers=1
	)


BATCH_SIZE = 8

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

Посмотрим на пример одного батча из итератора `train_data_loader`.



In [ ]:
data = next(iter(train_data_loader))
print(f'Батч:\n{data.keys()}\n')
print(f"Предложения в батче:\n{data['review_text']}\n")
print(f"Индексы токенов:\n{data['input_ids'].shape}\nbatch size x max len\n")
print(f"Маски внимания:\n{data['attention_mask'].shape}\nbatch size x max len\n")
print(f"Метки классов:\n{data['targets']}")
print(data['targets'].shape)

## Загрузка и создание модели

### Загрузка предобученной модели

In [ ]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model

Попробуем использовать эту модель. Применим её к токенизированному предложению. Модель принимает индексы токенов и маску внимания.

В переменную `hidden_states` записаны скрытые состояние слоя эмбеддингов и всех слоев энкодера (векторы каждого токена в предложении), в переменную `last_hidden_state` — скрытые состояния последнего слоя энкодера модели. Переменная `pooled_output` содержит выход линейного слоя модели — контекстный вектор для токена [CLS].

In [ ]:
print(f"Токены:\n{tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])}")
print(f"Индексы токенов:\n{encoding['input_ids'][0]}")
print(f"Маска внимания:\n{encoding['attention_mask']}")

last_hidden_state, pooled_output, hidden_states = bert_model(
	input_ids=encoding['input_ids'],
	attention_mask=encoding['attention_mask'],
	output_hidden_states=True,
	return_dict=False)

print(f'\nВсего скрытых состояний: {len(hidden_states)}')
print(f'Размер скрытых состояний последнего слоя: {last_hidden_state.shape}')
print(f'Размер выхода линейного слоя: {pooled_output.shape}')

### Анализ контекстных векторов

Посмотрим на векторы модели BERT для некоторых слов в предложениях.

In [ ]:
# text1 = df_marked.loc[16]['Reviews']
# text2 = df_marked.loc[338]['Reviews']
# test_word1, test_word2, test_word3 = 'курс', 'любая', 'практики'
# print(text1, len(text1))
# print(text2, len(text2))

In [ ]:
# encoding = tokenizer.encode_plus(
# 	text1,
# 	max_length=60,
# 	add_special_tokens=True,
# 	return_token_type_ids=False,
# 	padding='max_length',
# 	return_attention_mask=True,
# 	return_tensors='pt',
# 	truncation=True
# )
# print(f"Токенизированное предложение: {tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])}")
# 
# position1 = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]).index(test_word1)
# print(f'Позиция слова "{test_word1}": {position1}')
# 
# last_hidden_state, pooled_output = bert_model(
# 	input_ids=encoding['input_ids'],
# 	attention_mask=encoding['attention_mask'],
# 	return_dict=False)
# emb1 = last_hidden_state[0, position1, :]
# print(f'Размер вектора "{test_word1}": {emb1.shape}')

In [ ]:
# encoding2 = tokenizer.encode_plus(
# 	text2,
# 	max_length=60,
# 	add_special_tokens=True,
# 	return_token_type_ids=False,
# 	padding='max_length',
# 	return_attention_mask=True,
# 	return_tensors='pt',
# 	truncation=True
# )
# 
# print(f"Токенизированное предложение: {tokenizer.convert_ids_to_tokens(encoding2['input_ids'][0])}")
# position2 = tokenizer.convert_ids_to_tokens(encoding2['input_ids'][0]).index(test_word2)
# print(f'Позиция слова "{test_word2}": {position2}')
# position3 = tokenizer.convert_ids_to_tokens(encoding2['input_ids'][0]).index(test_word3)
# print(f'Позиция слова "{test_word3}": {position3}')
# 
# last_hidden_state2, pooled_output2 = bert_model(
# 	input_ids=encoding2['input_ids'],
# 	attention_mask=encoding2['attention_mask'],
# 	return_dict=False)
# 
# emb2 = last_hidden_state2[0, position2, :]
# emb3 = last_hidden_state[0, position3, :]
# print(f'Размер вектора "{test_word2}": {emb2.shape}')
# print(f'Размер вектора "{test_word3}": {emb3.shape}')

Посчитаем косинусное расстояние.

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# 
# print(
# 	f'Косинусное расстояние между словами "{test_word1}" и "{test_word2}": {cosine_similarity([emb1.detach().cpu().numpy()], [emb2.detach().cpu().numpy()])[0][0]}')
# print(
# 	f'Косинусное расстояние между словами "{test_word1}" и "{test_word3}": {cosine_similarity([emb1.detach().cpu().numpy()], [emb3.detach().cpu().numpy()])[0][0]}')

### Создание модели для классификации

Создадим класс `SentimentClassifier` на основе модели BERT.


In [ ]:
from torch import nn
from transformers import AutoModel


class ReviewsClassifier(nn.Module):

	def __init__(self, n_classes):
		super().__init__()
		self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
		self.drop = nn.Dropout(p=0.3)

		# Создаем отдельный линейный слой для каждого класса
		self.classifiers = nn.ModuleList([
			nn.Linear(self.bert.config.hidden_size, 1) for _ in range(n_classes)
		])

	def forward(self, input_ids, attention_mask):
		_, pooled_output = self.bert(
			input_ids=input_ids,
			attention_mask=attention_mask,
			return_dict=False)

		output = self.drop(pooled_output)

		# Применяем каждый линейный слой к одному и тому же входу, но независимо
		outputs = [torch.sigmoid(classifier(output)) for classifier in self.classifiers]

		# Соединяем все предсказания в один тензор
		return torch.cat(outputs, dim=1)

In [ ]:
n_classes = len(topic_keywords)
model = ReviewsClassifier(n_classes)
model

Перенесем на видеокарту один из батчей для примера и применим модель к данным.

In [ ]:
# input_ids = data['input_ids'].to(device)
# attention_mask = data['attention_mask'].to(device)
# 
# print(f'Индексы токенов:\n{input_ids.shape}\nbatch size x seq length')
# print(f'Маска внимания:\n{attention_mask.shape}\nbatch size x seq length')
# 
# output = model(input_ids, attention_mask)
# print(f'\nВыход модели:\n{output}')
# print(f'Размер:\n{output.shape}\nbatch size x num classes')

## Обучение, валидация и тестирование модели

### Обучение и валидация

Для дообучения модели будем использовать оптимизатор AdaW из библиотеки Transformers.

Авторы модели BERT рекомендуют использовать следующие параметры для дообучения модели:
- Размер батча: 16, 32
- Скорость обучения (с оптимизатором Adam): 5e-5, 3e-5, 2e-5
- Количество эпох: 2, 3, 4

Дообучение может происходить двумя способами:
- меняются веса на всех слоях (`requires_grad=True`);
- часть весов замораживается (`requires_grad=False`), для оставшихся слоев веса меняются (`requires_grad=True`).

По умолчанию для всех весов `requires_grad=True`. Чтобы заморозить веса, нужно установить параметр `requires_grad=False`. Заморозим веса для первых 5 слоев энкодера.

In [ ]:
print(f'До заморозки:\n {list(model.bert.encoder.layer[4].parameters())[0]}')

for layer_id in range(5):
	for param in list(model.bert.encoder.layer[layer_id].parameters()):
		param.requires_grad = False

print(f'\nПосле заморозки:\n {list(model.bert.encoder.layer[4].parameters())[0]}')

Зададим количество эпох и скорость обучения. Будем использовать планировщик (`scheduler`), он регулирует скорость обучения: первые несколько шагов (`num_warmup_steps`) она может увеличиваться, а потом уменьшается. Также определим функцию потерь.

In [ ]:
import os

EPOCHS = 1
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import AdamW, get_linear_schedule_with_warmup

# Обучение всех слоев
#optimizer = AdamW(model.parameters(), lr=2e-5) # обучение всех слоев
# Для обучения только незамороженных слоев нужно установить фильтр
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
	optimizer,
	num_warmup_steps=0,
	num_training_steps=total_steps
)

loss_fn = nn.BCELoss()

Реализуем функцию для одной эпохи обучения.

In [ ]:
import numpy as np


def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
	model.train()
	losses = []
	correct_predictions = 0

	for batch in tqdm(data_loader):
		input_ids = batch["input_ids"].to(device)
		attention_mask = batch["attention_mask"].to(device)
		targets = batch["targets"].to(device).float()  # Преобразование targets в float

		optimizer.zero_grad()

		outputs = model(input_ids=input_ids, attention_mask=attention_mask)

		loss = loss_fn(outputs, targets)
		loss.backward()
		optimizer.step()
		scheduler.step()

		# Применяем порог 0.5 для бинаризации предсказаний
		preds = (outputs > 0.5).int()

		# Сравниваем предсказания с целями по каждому классу
		correct_predictions += torch.sum(preds == targets).item()
		losses.append(loss.item())

	avg_loss = np.mean(losses)

	# Количество всех предсказаний равно числу примеров, умноженному на количество классов
	accuracy = correct_predictions / (n_examples * targets.shape[1])
	return accuracy, avg_loss

Также реализуем функцию для валидации.

In [ ]:

def eval_model(model, data_loader, loss_fn, device, n_examples):
	model.eval()
	losses = []
	correct_predictions = 0

	with torch.no_grad():
		for batch in data_loader:
			input_ids = batch["input_ids"].to(device)
			attention_mask = batch["attention_mask"].to(device)
			targets = batch["targets"].to(device).float()  # Преобразование targets в float

			outputs = model(input_ids=input_ids, attention_mask=attention_mask)
			loss = loss_fn(outputs, targets)

			# Применяем порог 0.5 для бинаризации предсказаний
			preds = (outputs > 0.5).int()

			# Сравниваем предсказания с целями по каждому классу
			correct_predictions += torch.sum(preds == targets).item()

			losses.append(loss.item())

	avg_loss = np.mean(losses)

	# Количество всех предсказаний равно числу примеров, умноженному на количество классов
	accuracy = correct_predictions / (n_examples * targets.shape[1])

	return accuracy, avg_loss

Используя эти две функции, реализуем процедуру дообучения модели.

In [ ]:
import os
import torch

path = '../weights/saved_weights.pt'
model = model.to(device)

# Проверяем, существует ли директория и создаем её, если она не существует
directory = os.path.dirname(path)
if not os.path.exists(directory):
	os.makedirs(directory)

if os.path.exists(path):
	print("Файл с весами найден. Загружаем веса вместо обучения.")
	model.load_state_dict(torch.load(path))
	history = None
else:
	# Если файла с весами нет, то обучаем модель
	from collections import defaultdict

	history = defaultdict(list)

	for epoch in range(EPOCHS):
		print(f'Epoch {epoch + 1}/{EPOCHS}')
		print('-' * 10)

		train_acc, train_loss = train_epoch(
			model,
			train_data_loader,
			loss_fn,
			optimizer,
			device,
			scheduler,
			len(df_train)
		)
		torch.cuda.empty_cache()
		print(f'Train loss {train_loss} accuracy {train_acc}')
		val_acc, val_loss = eval_model(
			model,
			val_data_loader,
			loss_fn,
			device,
			len(df_val)
		)
		torch.cuda.empty_cache()

		print(f'Val loss {val_loss} accuracy {val_acc}')

		history['train_acc'].append(train_acc)
		history['train_loss'].append(train_loss)
		history['val_acc'].append(val_acc)
		history['val_loss'].append(val_loss)

	torch.save(model.state_dict(), path)
	torch.cuda.empty_cache()

Можем сравнить точность на обучающей и валидационной выборке.

In [ ]:
if history is not None:  # Если модель обучалась, то можем сравнить точность эпох (не имеет смысла, если установлена 1 эпоха обучения) 
	import matplotlib.pyplot as plt

	plt.figure(figsize=(12, 6))

	# График точности (accuracy)
	plt.subplot(1, 2, 1)
	plt.plot(history['train_acc'], label='Train Accuracy', marker='o')
	plt.plot(history['val_acc'], label='Validation Accuracy', marker='o')
	plt.title('Accuracy Over Epochs')
	plt.xlabel('Epoch')
	plt.ylabel('Accuracy')
	plt.legend()
	plt.ylim([0, 1])
	plt.grid(True)

	# График потерь (loss)
	plt.subplot(1, 2, 2)
	plt.plot(history['train_loss'], label='Train Loss', marker='o', color='r')
	plt.plot(history['val_loss'], label='Validation Loss', marker='o', color='b')
	plt.title('Loss Over Epochs')
	plt.xlabel('Epoch')
	plt.ylabel('Loss')
	plt.legend()
	plt.grid(True)

	plt.tight_layout()
	plt.show()

### Тестирование

Подсчитаем точность (accuracy) модели на тестовой выборке.

In [ ]:
test_acc, _ = eval_model(
	model,
	test_data_loader,
	loss_fn,
	device,
	len(df_test)
)

test_acc

### Предсказание на произвольных текстах

Теперь нам осталось научиться использовать нашу модель для предсказания класса любого текста.

 Придумайте свой отзыв и проверьте работу модели.

In [ ]:
review_text = """Этот курс оставил у меня только положительные впечатления благодаря тому, как были поданы материалы. Ведущий демонстрировал высокий уровень знаний, что помогало глубже понять сложные темы. Каждый новый раздел был структурирован логично, с понятными примерами и практическими заданиями. Особое внимание уделялось важным деталям, что значительно облегчало усвоение информации. Рекомендую этот курс всем."""

Для использования нашей модели нам следует токенизировать текст соответствующим образом.

In [ ]:
encoded_review = tokenizer.encode_plus(
	review_text,
	max_length=MAX_LEN,
	add_special_tokens=True,
	return_token_type_ids=False,
	padding='max_length',
	return_attention_mask=True,
	return_tensors='pt',
	truncation=True
)

Теперь получим предсказания нашей модели.

In [ ]:
input_ids = encoded_review['input_ids']
attention_mask = encoded_review['attention_mask']

with torch.no_grad():
	output = model(input_ids.to(device), attention_mask.to(device))
torch.cuda.empty_cache()

# Применяем пороговое значение для классификации
predictions = (output > 0.5)
predictions = predictions.to(torch.int)

print(f'Review text: {review_text}')
print(f'Prediction  : {predictions}')
print('Предсказанные темы: {}'.format({topic_keywords[i] for i, val in enumerate(predictions.tolist()[0]) if val}))
print(output)

In [ ]:
import torch


# Функция для получения предсказания для одного отзыва
def predict_review(review_text):
	encoded_review = tokenizer.encode_plus(
		review_text,
		max_length=MAX_LEN,
		add_special_tokens=True,
		return_token_type_ids=False,
		padding='max_length',
		return_attention_mask=True,
		return_tensors='pt',
		truncation=True
	)

	input_ids = encoded_review['input_ids'].to(device)
	attention_mask = encoded_review['attention_mask'].to(device)

	with torch.no_grad():
		output = model(input_ids, attention_mask)

	torch.cuda.empty_cache()

	predictions = (output > 0.5).to(torch.int)

	return predictions.tolist()[0]


In [ ]:
df_t = pd.read_csv('../data/test_reviews.csv', index_col=0)

# Проход по всему DataFrame и предсказание меток для каждого отзыва
for index, row in df_t.iterrows():
	review_text = row['Reviews']
	predictions = predict_review(review_text)

	# Записываем предсказания в соответствующие столбцы
	df_t.at[index, 'практика'] = predictions[0]
	df_t.at[index, 'теория'] = predictions[1]
	df_t.at[index, 'преподаватель'] = predictions[2]
	df_t.at[index, 'технологии'] = predictions[3]
	df_t.at[index, 'актуальность'] = predictions[4]

# Теперь DataFrame df_t содержит обновленные метки для каждого отзыва
print(df_t)

In [ ]:
df_t.to_csv('../data/answer_bert.csv')